必要ModuleをImport

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import math

import random
import time
import pickle
import tqdm
from collections import Counter

from torch.utils.data import Dataset
import numpy as np

from utils import GELU, PositionwiseFeedForward, LayerNorm, SublayerConnection, LayerNorm

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import FloatProgress
from IPython.display import display, clear_output

read_dirでディレクトリを指定<br>
このディレクトリ内のコーパスを読み込む<br>
vocabファイルもここに作成される

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#read_dir = './data/single/'
read_dir = './data/merged_bert/'
#output_dir = './output/single/'
output_dir = './output/merged_bert/'
processed_train_txt = read_dir + 'train_X.txt'
processed_valid_txt = read_dir + 'valid_X.txt'
processed_test_txt = read_dir + 'test_X.txt'

Attentionセルを定義する

In [3]:
class Attention(nn.Module):
    """
    Scaled Dot Product Attention
    """
    def forward(self, query, key, value, mask=None, dropout=None):
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.size(-1))

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        p_attn = F.softmax(scores, dim=-1)

        if dropout is not None:
            p_attn = dropout(p_attn)

        return torch.matmul(p_attn, value), p_attn


Multi Head Attentionを定義する

In [4]:
class MultiHeadedAttention(nn.Module):

    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0

        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h

        self.linear_layers = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(3)])
        self.output_linear = nn.Linear(d_model, d_model)
        self.attention = Attention()

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        query, key, value = [l(x).view(batch_size, -1, self.h, self.d_k).transpose(1, 2) for l, x in zip(self.linear_layers, (query, key, value))]

        x, attn = self.attention(query, key, value, mask=mask, dropout=self.dropout)

        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.h * self.d_k)

        return self.output_linear(x)


Transformerを定義する

In [5]:
class TransformerBlock(nn.Module):
    def __init__(self, hidden, attn_heads, feed_forward_hidden, dropout):
        super().__init__()
        self.attention = MultiHeadedAttention(h=attn_heads, d_model=hidden, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model=hidden, d_ff=feed_forward_hidden, dropout=dropout)
        self.input_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.output_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, mask):
        x = self.input_sublayer(x, lambda _x: self.attention.forward(_x, _x, _x, mask=mask))
        x = self.output_sublayer(x, self.feed_forward)
        return self.dropout(x)


BERTクラスを定義する

In [6]:
class BERT(nn.Module):

    def __init__(self, vocab_size, hidden=768, n_layers=12, attn_heads=12, dropout=0.1):
        super().__init__()
        self.hidden = hidden
        self.n_layers = n_layers
        self.attn_heads = attn_heads
        self.feed_forward_hidden = hidden * 4
        # embedding for BERT
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=hidden, dropout=dropout)

        self.transformer_blocks = nn.ModuleList([TransformerBlock(hidden, attn_heads, hidden * 4, dropout) for _ in range(n_layers)])

    def forward(self, x):
        # xの中で0以上は1, 0未満は0として, maskテンソルを作る
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)
        x = self.embedding(x)

        for transformer in self.transformer_blocks:
            x = transformer.forward(x, mask)
        return x


BERTのEmbedding層を定義する

In [7]:
class TokenEmbedding(nn.Embedding):
    def __init__(self, vocab_size, embed_size=512):
        super().__init__(vocab_size, embed_size, padding_idx=0)

class PositionalEmbedding(nn.Module):

    def __init__(self, d_model, max_len=512):
        super().__init__()

        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)).float().exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class BERTEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_size, dropout=0.1):
        super().__init__()
        self.token = TokenEmbedding(vocab_size=vocab_size, embed_size=embed_size)
        self.position = PositionalEmbedding(d_model=self.token.embedding_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.embed_size = embed_size

    #def forward(self, sequence, segment_label):
    def forward(self, sequence):
        x = self.token(sequence) + self.position(sequence)
        return self.dropout(x)


学習用にマスク予測の層を追加する<br>
Next Sentence Prediction用のクラスは削除してある

In [8]:
class BERTLM(nn.Module):
    def __init__(self, bert: BERT, vocab_size):
        super().__init__()
        self.bert = bert
        self.mask_lm = MaskedLanguageModel(self.bert.hidden, vocab_size)

    def forward(self, x):
        x = self.bert(x)
        return self.mask_lm(x)

class MaskedLanguageModel(nn.Module):
    def __init__(self, hidden, vocab_size):
        super().__init__()
        self.linear = nn.Linear(hidden, vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x))

In [9]:
class BERTHybrid(nn.Module):
    def __init__(self, bert: BERT, head):
        super().__init__()
        self.bert = bert
        self.epd = ErrorPositionDetectionHead(self.bert.hidden, head)
        self.esd = ErrorSentenceDetectionHead(head)
    def forward(self, x):
        x = self.bert(x)
        x = self.epd(x)
        return x, self.esd(x) 

class ErrorSentenceDetectionHead(nn.Module):
    def __init__(self, hidden):
        super().__init__()
        self.linear = nn.Linear(hidden, 2)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x[:, 0]))
    
class ErrorPositionDetectionHead(nn.Module):
    def __init__(self, hidden, head):
        super().__init__()
        self.linear = nn.Linear(hidden, head)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x))

BERT用のVocabを生成するクラスを定義する

In [10]:
import pickle
import tqdm
from collections import Counter

class TorchVocab(object):
    """
    :property freqs: collections.Counter, コーパス中の単語の出現頻度を保持するオブジェクト
    :property stoi: collections.defaultdict, string → id の対応を示す辞書
    :property itos: collections.defaultdict, id → string の対応を示す辞書
    """
    def __init__(self, counter, max_size=None, min_freq=1, specials=['<pad>', '<oov>'],
                 vectors=None, unk_init=None, vectors_cache=None):
        """
        :param coutenr: collections.Counter, データ中に含まれる単語の頻度を計測するためのcounter
        :param max_size: int, vocabularyの最大のサイズ. Noneの場合は最大値なし. defaultはNone
        :param min_freq: int, vocabulary中の単語の最低出現頻度. この数以下の出現回数の単語はvocabularyに加えられない.
        :param specials: list of str, vocabularyにあらかじめ登録するtoken
        :param vecors: list of vectors, 事前学習済みのベクトル. ex)Vocab.load_vectors
        """
        self.freqs = counter
        counter = counter.copy()
        min_freq = max(min_freq, 1)

        self.itos = list(specials)
        # special tokensの出現頻度はvocabulary作成の際にカウントされない
        for tok in specials:
            del counter[tok]

        max_size = None if max_size is None else max_size + len(self.itos)

        # まず頻度でソートし、次に文字順で並び替える
        words_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
        words_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)
        
        # 出現頻度がmin_freq未満のものはvocabに加えない
        for word, freq in words_and_frequencies:
            if freq < min_freq or len(self.itos) == max_size:
                break
            self.itos.append(word)

        # dictのk,vをいれかえてstoiを作成する
        self.stoi = {tok: i for i, tok in enumerate(self.itos)}

        self.vectors = None
        if vectors is not None:
            self.load_vectors(vectors, unk_init=unk_init, cache=vectors_cache)
        else:
            assert unk_init is None and vectors_cache is None

    def __eq__(self, other):
        if self.freqs != other.freqs:
            return False
        if self.stoi != other.stoi:
            return False
        if self.itos != other.itos:
            return False
        if self.vectors != other.vectors:
            return False
        return True

    def __len__(self):
        return len(self.itos)

    def vocab_rerank(self):
        self.stoi = {word: i for i, word in enumerate(self.itos)}

    def extend(self, v, sort=False):
        words = sorted(v.itos) if sort else v.itos
        for w in words:
            if w not in self.stoi:
                self.itos.append(w)
                self.stoi[w] = len(self.itos) - 1


class Vocab(TorchVocab):
    def __init__(self, counter, max_size=None, min_freq=1):
        self.pad_index = 0
        self.unk_index = 1
        self.eos_index = 2
        self.sos_index = 3
        self.mask_index = 4
        super().__init__(counter, specials=["<pad>", "<unk>", "<eos>", "<sos>", "<mask>"], max_size=max_size, min_freq=min_freq)

    # override用
    def to_seq(self, sentece, seq_len, with_eos=False, with_sos=False) -> list:
        pass

    # override用
    def from_seq(self, seq, join=False, with_pad=False):
        pass

    @staticmethod
    def load_vocab(vocab_path: str) -> 'Vocab':
        with open(vocab_path, "rb") as f:
            return pickle.load(f)

    def save_vocab(self, vocab_path):
        with open(vocab_path, "wb") as f:
            pickle.dump(self, f)


# テキストファイルからvocabを作成する
class WordVocab(Vocab):
    def __init__(self, texts, max_size=None, min_freq=1):
        print("Building Vocab")
        counter = Counter()
        for line in texts:
            if isinstance(line, list):
                words = line
            else:
                words = line.replace("\n", "").replace("\t", "").split()

            for word in words:
                counter[word] += 1
        super().__init__(counter, max_size=max_size, min_freq=min_freq)

    def to_seq(self, sentence, seq_len=None, with_eos=False, with_sos=False, with_len=False):
        if isinstance(sentence, str):
            sentence = sentence.split()

        seq = [self.stoi.get(word, self.unk_index) for word in sentence]

        if with_eos:
            seq += [self.eos_index]  # this would be index 1
        if with_sos:
            seq = [self.sos_index] + seq

        origin_seq_len = len(seq)

        if seq_len is None:
            pass
        elif len(seq) <= seq_len:
            seq += [self.pad_index for _ in range(seq_len - len(seq))]
        else:
            seq = seq[:seq_len]

        return (seq, origin_seq_len) if with_len else seq

    def from_seq(self, seq, join=False, with_pad=False):
        words = [self.itos[idx]
                 if idx < len(self.itos)
                 else "<%d>" % idx
                 for idx in seq
                 if not with_pad or idx != self.pad_index]

        return " ".join(words) if join else words

    @staticmethod
    def load_vocab(vocab_path: str) -> 'WordVocab':
        with open(vocab_path, "rb") as f:
            return pickle.load(f)


def build(corpus_path, output_path, vocab_size=None, encoding='utf-8', min_freq=1):
    with open(corpus_path, "r", encoding=encoding) as f:
        vocab = WordVocab(f, max_size=vocab_size, min_freq=min_freq)

    print("VOCAB SIZE:", len(vocab))
    vocab.save_vocab(output_path)

In [11]:
import datetime
dt_now = str(datetime.datetime.now()).replace(' ', '')
dt_now = dt_now.replace(':','_')

In [12]:
from torch.utils.data import DataLoader
class HybridTrainer:
    def __init__(self, bert: BERT, train_dataloader: DataLoader, 
                 valid_dataloader: DataLoader = None,
                 lr: float = 1e-4, betas=(0.9, 0.999), weight_decay: float = 0.01,
                 with_cuda: bool = True, log_freq: int = 10, head = 9):
        # GPU環境において、GPUを指定しているかのフラグ
        cuda_condition = torch.cuda.is_available() and with_cuda
        self.device = torch.device("cuda:0" if cuda_condition else "cpu")

        self.bert = bert
        self.model = BERTHybrid(self.bert, head).to(self.device)

        if torch.cuda.device_count() > 1:
            print("Using %d GPUS for BERT" % torch.cuda.device_count())
            self.model = nn.DataParallel(self.model)

        self.train_data = train_dataloader
        self.valid_data = valid_dataloader

        self.optim = Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)

        # masked_token予測のためのLoss関数を設定
        self.criterion = nn.NLLLoss()
        self.log_freq = log_freq
        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))
        
        self.train_losses = []
        self.valid_losses = []
        self.train_epd_accs = []
        self.valid_epd_accs = []
        self.train_esd_accs = []
        self.valid_esd_accs = []
        
        self.train_epd_accs_epoch = []
        self.valid_epd_accs_epoch = []
        self.train_esd_accs_epoch = []
        self.valid_esd_accs_epoch = []

    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def valid(self, epoch):
        self.iteration(epoch, self.valid_data, train=False)

    def iteration(self, epoch, data_loader, train=True):
        str_code = "train" if train else "valid"
        data_iter = tqdm.tqdm(enumerate(data_loader), desc="EP_%s:%d" % (str_code, epoch), total=len(data_loader), bar_format="{l_bar}{r_bar}")
        avg_loss = 0.0
        total_epd_correct = 0
        total_esd_correct = 0
        total_element = 0
        for i, data in data_iter:
            data = {key: value.to(self.device) for key, value in data.items()}
            epd_output, esd_output = self.model.forward(data["bert_input"])
            epd_loss = self.criterion(epd_output.transpose(1, 2), data["token_label"])
            esd_loss = self.criterion(esd_output, data["label"])
            
            loss = epd_loss + esd_loss
            # training時のみ,backwardとoptimizer更新を行う
            if train:
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
            
            epd_correct = epd_output.argmax(dim=-1).eq(data["token_label"]).sum().item()
            esd_correct = esd_output.argmax(dim=-1).eq(data["label"]).sum().item()
            avg_loss += loss.item()
            total_epd_correct += epd_correct
            total_esd_correct += esd_correct
            total_element += data["label"].nelement()
            post_fix = {
                "epoch": epoch,
                "iter": i,
                "avg_loss": avg_loss / (i + 1),
                "avg_epd_acc": total_epd_correct / (total_element*128) * 100,
                "avg_esd_acc": total_esd_correct / total_element * 100,                
                "loss": loss.item()
            }

            if i % self.log_freq == 0:
                data_iter.write(str(post_fix))

        print("EP%d_%s, avg_loss=" % (epoch, str_code), avg_loss / len(data_iter),  
              "total_epd_acc=", total_epd_correct / (total_element*128) * 100,
                "total_esd_acc=", total_esd_correct * 100.0 / total_element)
        if train:
            self.train_losses.append(avg_loss / len(data_iter))
            self.train_epd_accs.append(total_epd_correct / (total_element*128) * 100 )
            self.train_esd_accs.append(total_esd_correct * 100.0 / total_element)
        else:
            self.valid_losses.append(avg_loss / len(data_iter))
            self.valid_epd_accs.append(total_epd_correct / (total_element*128) * 100)
            self.valid_esd_accs.append(total_esd_correct * 100.0 / total_element)
        
    def save_bert(self, epoch, file_path=output_dir + "bert_model"):
        output_path = file_path + ".ep%d" % epoch
        torch.save(self.bert.state_dict(), output_path)
        self.bert.to(self.device)
        print("EP:%d Model Saved on:" % epoch, output_path)
        return output_path
    
    def save_pretrain(self, epoch, file_path=output_dir + "mlm_model"):
        self.train_epd_accs_epoch.append(self.train_epd_accs[-1])
        self.train_esd_accs_epoch.append(self.train_esd_accs[-1])
        self.train_epd_accs_epoch.append(self.valid_epd_accs[-1])
        self.train_esd_accs_epoch.append(self.valid_esd_accs[-1])
        state = {
            'epoch' : epoch,
            'state_dict' : self.model.module.state_dict(),
            'optimizer' : self.optim.state_dict(),
            'train_epd_acc': self.train_epd_accs_epoch,
            'train_esd_acc': self.train_esd_accs_epoch,
            'valid_epd_acc': self.valid_epd_accs_epoch,
            'valid_esd_acc': self.valid_esd_accs_epoch
        }
        output_path = file_path + ".ep%d" % epoch
        torch.save(state, output_path)
        self.bert.to(self.device)
        print("EP:%d Model Saved on:" % epoch, output_path)
        return output_path
 
    def load_bert(self, file_path):
        self.bert.state_dict(torch.load(file_path))
    
    def load_pretrain(self, file_path):
        state = torch.load(file_path)
        #state = self.fix_model_state_dict(state)
        self.model.module.load_state_dict(state['state_dict'])
        self.optim.load_state_dict(state['optimizer'])
        self.train_epd_accs_epoch = state['train_epd_acc']
        self.train_esd_accs_epoch = state['train_esd_acc']
        self.valid_epd_accs_epoch = state['valid_epd_acc']
        self.valid_esd_accs_epoch = state['valid_esd_acc']
        return state

In [13]:
# 訓練用パラメタを定義する
vocab_path= read_dir + 'vocab.txt'


hidden=256 #768
layers=2 #12 2
attn_heads=4 #12 4
seq_len=128

batch_size=128
epochs=50
num_workers=0
with_cuda=True
log_freq=100
corpus_lines=None

lr=1e-3
adam_weight_decay=0.00
adam_beta1=0.9
adam_beta2=0.999

dropout=0.0

min_freq=7

corpus_path=processed_train_txt
label_path=None

# 最初から学習する場合は'first'、学習を続きから再開する場合は'continue'
#train_mode = 'first' 
train_mode = 'continue' 

In [14]:
print("Loading Vocab", vocab_path)

vocab = WordVocab.load_vocab(vocab_path)
from dataset import MixDataset 

print("Loading Train Dataset", processed_train_txt)
train_dataset = MixDataset(processed_train_txt, vocab, seq_len=seq_len, label_path=label_path, corpus_lines=corpus_lines, is_train=True)

print("Loading Valid Dataset", processed_valid_txt)
valid_dataset = MixDataset(processed_valid_txt, vocab, seq_len=seq_len, label_path=label_path) if processed_valid_txt is not None else None


print("Creating Dataloader")
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers) if processed_valid_txt is not None else None

Loading Vocab ./data/merged_bert/vocab.txt
Loading Train Dataset ./data/merged_bert/train_X.txt
Loading Valid Dataset ./data/merged_bert/valid_X.txt
Creating Dataloader


In [15]:
print("Building BERT model")
bert = BERT(len(vocab), hidden=hidden, n_layers=layers, attn_heads=attn_heads, dropout=dropout)
trainer = HybridTrainer(bert, train_dataloader=train_data_loader, valid_dataloader=valid_data_loader,
                        lr=lr, betas=(adam_beta1, adam_beta2), weight_decay=adam_weight_decay,
                        with_cuda=with_cuda, log_freq=log_freq, head=9)

Building BERT model
Using 2 GPUS for BERT
Total Parameters: 2761501


In [16]:
output_model_path = output_dir + 'mix/hybrid/small'

if train_mode == 'first':
    trainer.load_bert(output_dir + 'bert/mid/bert.ep16')
    trained_epoch = 0

else:
    state = trainer.load_pretrain(output_model_path + '.ep26')
    trained_epoch = state['epoch'] +1
    print(trained_epoch)

27


In [17]:
print("Training Start")
for epoch in range(trained_epoch, epochs):
    epoch_start = time.time()
    trainer.train(epoch)
    # Model Save
    trainer.valid(epoch)
    trainer.save_pretrain(epoch, output_model_path)
    print(time.time() - epoch_start)

Training Start


EP_train:27:   0%|| 0/50252 [00:00<?, ?it/s]C:\Users\is0368pi\.conda\envs\nlp\lib\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


{'epoch': 27, 'iter': 0, 'avg_loss': 0.29316115379333496, 'avg_epd_acc': 96.795654296875, 'avg_esd_acc': 90.625, 'loss': 0.29316115379333496}


{'epoch': 27, 'iter': 100, 'avg_loss': 0.32064396024930597, 'avg_epd_acc': 96.85022335241337, 'avg_esd_acc': 90.3697400990099, 'loss': 0.341367244720459}


{'epoch': 27, 'iter': 200, 'avg_loss': 0.32135508198346663, 'avg_epd_acc': 96.85040374300374, 'avg_esd_acc': 90.38013059701493, 'loss': 0.39842116832733154}


...

{'epoch': 27, 'iter': 6100, 'avg_loss': 0.3205286975430379, 'avg_epd_acc': 96.66087074448605, 'avg_esd_acc': 90.63716501393215, 'loss': 0.33363163471221924}


{'epoch': 27, 'iter': 6200, 'avg_loss': 0.3205940580559323, 'avg_epd_acc': 96.66088776633809, 'avg_esd_acc': 90.63293722786648, 'loss': 0.3263503909111023}


EP_valid:27: 100%|| 6282/6282 [03:27<00:00, 30.29it/s]


EP27_valid, avg_loss= 0.3205240870316254 total_epd_acc= 96.66106136974206 total_esd_acc= 90.63791945473993


AttributeError: 'float' object has no attribute 'append'

In [ ]:
import matplotlib.pyplot as plt

import matplotlib.ticker as ticker
import numpy as np
plt.style.use('ggplot')

def showPlot(points):

    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    #loc = ticker.MultipleLocator(base=0.2)
    #ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()
    
def savePlot(points, figure_path):
    plt.switch_backend('Agg')
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.savefig(figure_path)

In [ ]:
def save_results():
    with open(output_model_path + "results.txt","w",encoding='utf-8') as f:
        f.write("----------Parameters list----------\n")
        f.write("hidden=" +str(hidden) + "\n")
        f.write("layers="+str(layers)+"\n")
        f.write("attn_heads="+str(attn_heads)+"\n")
        f.write("seq_len="+str(seq_len)+"\n")
        f.write("batch_size="+str(batch_size)+"\n")
        f.write("epochs="+str(epochs)+"\n")
        f.write("lr="+str(lr)+"\n")
        f.write("\n")
        
        f.write("-"*10+"Train Loss"+"-"*10+"\n")
        f.write(str(trainer.train_losses[-1]))
        f.write("\n")
        
        f.write("-"*10+"Valid Loss"+"-"*10+"\n")
        f.write(str(trainer.valid_losses[-1]))
        f.write("\n")
        
        f.write("-"*10+"Train EPD Acc"+"-"*10+"\n")
        f.write(str(trainer.train_epd_accs[-1]))
        f.write("\n")
        
        f.write("-"*10+"Train ESD Acc"+"-"*10+"\n")
        f.write(str(trainer.train_esd_accs[-1]))
        f.write("\n")
        
        f.write("-"*10+"Valid EPD Acc"+"-"*10+"\n")
        f.write(str(trainer.valid_epd_accs[-1]))
        f.write("-"*10+"Valid ESD Acc"+"-"*10+"\n")
        f.write(str(trainer.valid_esd_accs[-1]))
    savePlot(trainer.train_epd_accs, output_model_path + "train_epd_accs")
    savePlot(trainer.train_esd_accs, output_model_path + "train_esd_accs")
    savePlot(trainer.valid_epd_accs, output_model_path + "valid_epd_accs")
    savePlot(trainer.valid_esd_accs, output_model_path + "valid_esd_accs")
    savePlot(trainer.train_losses, output_model_path + "train_losses")
    savePlot(trainer.valid_losses, output_model_path + "valid_losses")
save_results()

TestデータからDataLoaderを作成

In [ ]:
test_dataset = MixDataset(processed_test_txt, vocab, seq_len=seq_len, label_path=label_path, is_train=True)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=num_workers, shuffle=False)

In [ ]:
def test_esd():
    trainer.model.eval()
    with torch.no_grad():
        for batch in test_loader:
            #batch = next(iter(test_loader))
            input_line = []
            text = batch['bert_input'][0]
            output = trainer.model(batch['bert_input'])
            for j in range(len(text)):
                input_line.append(vocab.itos[text[j].item()])
            #print("".join(input_line))
            print(batch['label'].item(), output.argmax(dim=-1).item())
test_esd()

In [ ]:
def test_epd():
    trainer.model.eval()
    with torch.no_grad():
        for batch in test_loader:
            #batch = next(iter(test_loader))
            input_line = []
            text = batch['bert_input'][0]
            output = trainer.model(batch['bert_input'])
            for j in range(len(text)):
                input_line.append(vocab.itos[text[j].item()])
            print("".join(input_line))
            #print(batch['token_label'][0], output.argmax(dim=-1))
            for j in range(128):
                predict = output[0,j].argmax(dim=-1).item()
                print(input_line[j], ':',predict)
            break

test_epd()

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('Agg')
import matplotlib.ticker as ticker
import numpy as np
plt.style.use('ggplot')

def showPlot(points, figure_path):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.savefig(figure_path)

In [ ]:
showPlot(trainer.train_losses, "./results/"+str(epochs)+"_train.png")
showPlot(trainer.valid_losses, "./results/"+str(epochs)+"_valid.png")